* `spikeflow` tool was written in mind with interpretability and ease of examining the spiking network's key internal variables. Convolution and Pooling was implemented using `Tensorflow` and most of the code was written using `NumPy`. Spike accumulation, thresholding and lateral inhibition can be implemented in a single graph of `Tensorflow` leaving STDP competition to be handled by `NumPy`. This is an extra notebook that shows how to code lateral inhibition in `Tensorflow`. This can replace the `NumPy` based lateral inhibition in `spikeflow`.

## Imports

In [1]:
import os, time
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import numpy as np
import tensorflow as tf
from copy import deepcopy
from tensorflow.python.client import timeline
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(tf.__version__)
tf

1.14.0


<module 'tensorflow' from '/home/ruthvik/.local/lib/python2.7/site-packages/tensorflow/__init__.pyc'>

## Open Session

In [3]:
sess= tf.InteractiveSession()
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()

## Hide code

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


## Generate data

In [4]:
pots_list = []
SpikesPerNeuronAllowed_list = []
np_pots_list = []
nofMaps = 100
x_dim = 25
y_dim = 25
for i in range(10):
    pots = np.random.rand(nofMaps,x_dim,y_dim)
    pots_list.append(pots)
    temp = np.random.choice([0, 1], size=(int(x_dim*y_dim),), p=[0.1, 0.9])
    SpikesPerNeuronAllowed = temp.reshape(x_dim,y_dim)
    SpikesPerNeuronAllowed_list.append(SpikesPerNeuronAllowed)
    np_pots_list.append(tf.transpose(pots_list[i],perm=[1,2,0]).eval())

## Lateral inhibition in Tensorflow

### Setup the graph

In [5]:
#with tf.name_scope('placeholders'):
voltages = tf.placeholder(dtype=tf.float64,shape=(nofMaps,x_dim,y_dim), name='voltages')
SpikesPerNeuronAllowed_tf = tf.placeholder(dtype=tf.int64,shape=(x_dim,y_dim), name='spk_allowed')

#with tf.name_scope('thresholding'):
thresholding = tf.where(tf.greater_equal(voltages,0.5), tf.ones(shape=(nofMaps,x_dim,y_dim),dtype=tf.int64), 
                        tf.zeros(shape=(nofMaps,x_dim,y_dim),dtype=tf.int64), name='thresh_op')
    
#with tf.name_scope('lateral_inhibition'):
########## TO KILL THE NEURONS THAT ALREADY SPIKED IN PREVIOUS TIMESTEPS ###############(METHOD 2)
suppress_prev_spikes = tf.multiply(thresholding,SpikesPerNeuronAllowed_tf,name='stg1_compt.')

########## TO FILTER SPIKES THAT ARE ALOWED TO SPIKES IN THIS TSTEP AND HAVE SPIKED THE EARLIEST
max_voltages = tf.reduce_max(voltages, axis=0,keepdims=True,name='max_val_at_xy_loc')
condition = tf.equal(voltages, max_voltages,name='max_xy_loc_indices')
first_spikes = tf.where(condition, tf.ones_like(voltages,dtype=tf.int64),
                        tf.zeros_like(voltages,dtype=tf.int64),name='1st_spk_calc')
allowed_first_spikes = tf.multiply(first_spikes,suppress_prev_spikes,name='stg2_compt.')

#with tf.name_scope('update_spksperneuronallowed'):
#############UPDATE THE SPIKESPERNEURONALLOWED TENSOR (METHOD1)#######
new_SpikesPerNeuronAllowed_graph = tf.multiply(SpikesPerNeuronAllowed_tf,(1-tf.reduce_sum(allowed_first_spikes,
                                        axis=0,keepdims=True)),name='updt_spk_allowed')

init_op = tf.global_variables_initializer()
sess.run(init_op,options=run_options, run_metadata=run_metadata)
#sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Run the graph

In [6]:
for i in range(10):
    pots = pots_list[i]
    #SpikesPerNeuronAllowed = deepcopy(SpikesPerNeuronAllowed_list[i])
    SpikesPerNeuronAllowed = SpikesPerNeuronAllowed_list[i]
    
#https://stackoverflow.com/questions/38619107/accessing-intermediate-results-from-a-tensorflow-graph
    t2 = time.time()
    new_SpikesPerNeuronAllowed,final_spikes=sess.run([new_SpikesPerNeuronAllowed_graph,allowed_first_spikes], 
       feed_dict={SpikesPerNeuronAllowed_tf:SpikesPerNeuronAllowed,voltages:pots},options=run_options, 
                                          run_metadata=run_metadata)
    #new_SpikesPerNeuronAllowed,final_spikes=sess.run([new_SpikesPerNeuronAllowed_graph,allowed_first_spikes], 
    #    feed_dict={SpikesPerNeuronAllowed_tf:SpikesPerNeuronAllowed,voltages:pots})
    print('time taken:{}'.format(time.time()-t2))
        
    print(SpikesPerNeuronAllowed.sum(),final_spikes.sum(),new_SpikesPerNeuronAllowed.sum())
    
    

time taken:0.014673948288
(566, 566, 0)
time taken:0.00195503234863
(581, 581, 0)
time taken:0.00211906433105
(569, 569, 0)
time taken:0.0044641494751
(567, 567, 0)
time taken:0.00220990180969
(574, 574, 0)
time taken:0.00200581550598
(562, 562, 0)
time taken:0.00174689292908
(566, 566, 0)
time taken:0.00165414810181
(564, 564, 0)
time taken:0.00165510177612
(561, 561, 0)
time taken:0.00178003311157
(556, 556, 0)


## Profiling

In [ ]:
n = 10        # Number of loops
r = 512     # Number of repetitions of each loop
p = 3        # Display precision
t1 = %timeit -n 10 -r 512 -p 6 -o pass; func = sess.run([new_SpikesPerNeuronAllowed_graph,allowed_first_spikes],\
                                                        feed_dict={SpikesPerNeuronAllowed_tf:SpikesPerNeuronAllowed,voltages:pots},options=run_options,\
                                                        run_metadata=run_metadata)

In [ ]:
print('mean of {} run times:{}'.format(len(t1.all_runs),np.mean(t1.all_runs)/n))
print('std of {} run times:{}'.format(len(t1.all_runs),np.std(t1.all_runs)/n))
print('max of {} run times:{}'.format(len(t1.all_runs),np.max(t1.all_runs)/n))
print('min of {} run times:{}'.format(len(t1.all_runs),np.min(t1.all_runs)/n))

### Save the graph and data
* To run the Tensorboard `cd` into the `path` and then execute ` python -m tensorboard.main --logdir=./`

In [ ]:
path = '/home/ruthvik/Desktop/Summer 2017/tf_graph_outputs/lateral_inh/'
writer = tf.summary.FileWriter(path, sess.graph,flush_secs=1)
writer.flush()

tl = timeline.Timeline(run_metadata.step_stats)
ctf = tl.generate_chrome_trace_format()
with open(path+'/timeline.json', 'w') as f:
    f.write(ctf)


writer.add_run_metadata(run_metadata, 'mysess')
writer.flush()
writer.close()
#sess.close()

## Numpy (Lateral Inhibition)

In [ ]:
def lateral_inhibition(pots,conv_spikes,spiked_neurons,SpikesPerNeuronAllowed):
        vbn = np.where(SpikesPerNeuronAllowed==0)
        conv_spikes[vbn[0],vbn[1],:]=0 #if a neuron in a position has spiked b4 don't let it spike 
        high_volts=np.zeros(pots.shape)
        idx = pots.argmax(axis=-1)
        high_volts[np.arange(pots.shape[0])[:,None],np.arange(pots.shape[1]),idx] = 1
        bvc = np.where(conv_spikes*high_volts==1)
        conv_spikes[bvc[0],bvc[1],bvc[2]]=1 #aross layers, only neurons with high pot get to keep spikes
        bvc1 = np.where(conv_spikes*high_volts!=1)
        conv_spikes[bvc1[0],bvc1[1],bvc1[2]]=0 #all other neurons that spiked with low pots dont get to spike
        spiked_neurons = np.where(conv_spikes==1)
        SpikesPerNeuronAllowed[spiked_neurons[0],spiked_neurons[1]]=0

        return conv_spikes, SpikesPerNeuronAllowed

### Run the Numpy function

In [ ]:
for i in range(10):
    pots = deepcopy(np_pots_list[i])
    SpikesPerNeuronAllowed = deepcopy(SpikesPerNeuronAllowed_list[i])
    t2 = time.time()
    final_spikes, new_SpikesPerNeuronAllowed = lateral_inhibition(pots,pots>=0.5,np.where(pots>=0.5),
                                                    SpikesPerNeuronAllowed)
    print('time taken:{}'.format(time.time()-t2))
    
    print(SpikesPerNeuronAllowed_list[i].sum(),final_spikes.sum(),new_SpikesPerNeuronAllowed.sum())
    